In [ ]:
# %pip uninstall keras -y
# %pip install keras==3.6.0
# %pip list

In [ ]:
%pip install keras==3.6.0
%pip install tensorflow
%pip install pandas
%pip install seaborn
%pip install scikit-learn
%pip install pretty_midi
%pip install music21
%pip install torch
%pip install torcheval


In [ ]:
import pandas as pd
import numpy as np
import pretty_midi
import pathlib
import collections
import datetime
import glob
import music21
import tensorflow as tf
import torch
from torcheval.metrics.text import Perplexity
import random


# Music Gen using LSTM model

This project will use a 4 layer keras LSTM model to predict notes based on training from a famous jazz recording captures that were converted to midi.

The notebook has 3 main sections.
* Training Data Preparation
* Model Definition and fit
* Predictions based on sample input

### Helper Functions

Some conversion and database i/o functions were split into a separate python file to help readability of this notebook.  This are loaded here.

In [ ]:
import importlib  # allows reload of my_functions
import my_functions

In [ ]:
importlib.reload(my_functions)

### This cell defines global constants that are used throughout the notebook 

In [ ]:
# Global things used throughout the notebook

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# length of trainig sequences
seq_length = 20

# Size of pitch vocab
vocab_size = 128

# Keys that will get extracted into the training set. This are the inputs to the model!
key_order = ['pitch', 'step', 'duration', 'tempo', 'instrument_num', 'key_num']

# Songs to exclude from training
#skip_list = random.sample(range(1, 456), 16)
skip_list = [ 311, 50, 66, 283, 95, 303, 284, 238, 297, 276, 416, 254, 346, 161, 125, 362]

# Skipped songs to be used for Validation
val_list = skip_list[:10]

# Skipped songs to be used for Testing
test_list = skip_list[10:]


model_save_file = 'PurePitchSave.keras'


### This cell will load training data using a SQL query and calculate some differential values

Training data is loaded into a Dataframe.  Many fields are just extracted from the wjazzd.db sqlite
database. Others are calculated on the dataframe once it is created.

Fields for each note from the database are created joining the tables "melody" and "solo_info"
* pitch - from melody.pitch midi note value (0-127)
* pitch_norm - from melody.pitch / 128 as pitch_norm  Normalized pitch.  (not used)
* start - from melody.onset as start  Start time of the note
* end - from melody.onset + melody.duration as end  End time of the note
* duration - from melody.duration duration of the note in seconds
* instrument - from solo_info.instrument string representing of the instrument used ie "cl" (clarinet)
* key - solo_info.key string value reprensenting the musical key ie: "Bb-maj"
* style - from solo_info.style string representing the song genre ie: "COOL" or "POSTBOP"
* tempo - from solo_info.avgtempo as tempo reprensented in BPM
* feel - from solo_info.rhythmfeel as feel string represening rhythmic feel ie "SWING" or "LATIN" 
* title - from solo_info.title,
* performer = from solo_info.performer string of the name of the artist on the solo

Once the data is loaded, these additional fields are create
* step - time between the previous note and this note
* interval - pitch difference between
* contour - interval abstraction based on my_functions.contour()



In [ ]:
# This loads up all the notes in the dataset
pitchInst = my_functions.extract_training_data()

### Turn to alpha numeric lables into numbers for training input
* instrument_num maps to the type of instrument used in the solo
* key_num maps to the musical key that the tune was in (Bb-maj for example)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
pitchInst['instrument_num'] = le.fit_transform(pitchInst['instrument']).astype(float)
pitchInst['key_num'] = le.fit_transform(pitchInst['key']).astype(float)


### Trim off extraneous notes from the training data and skip songs

For the model to train correctly, the training set must be an integral multiple of seq_length.


In [ ]:
n_notes, train_notes = my_functions.trim_data_sequence(
        pitchInst, 
        skip_list, 
        seq_length,
        key_order)

notes_ds = tf.data.Dataset.from_tensor_slices(train_notes)
notes_ds.element_spec

In [ ]:
val_length, val_notes = my_functions.select_data_sequences(pitchInst, 
        val_list, 
        seq_length,
        key_order)

val_notes_ds = tf.data.Dataset.from_tensor_slices(val_notes)
val_notes_ds.element_spec

In [ ]:
val_seq_ds = my_functions.create_sequences(val_notes_ds, key_order, seq_length, vocab_size)
val_seq_ds.__len__

In [ ]:
batch_size = 64
buffer_size = val_length #  - seq_length  # the number of items in the dataset
val_ds = (val_seq_ds
            .shuffle(buffer_size)
            .batch(batch_size, drop_remainder=True)
            .cache()
            .prefetch(tf.data.experimental.AUTOTUNE))

In [ ]:
seq_ds = my_functions.create_sequences(notes_ds, key_order, seq_length, vocab_size)
seq_ds.__len__

In [ ]:
batch_size = 64
buffer_size = n_notes #  - seq_length  # the number of items in the dataset
train_ds = (seq_ds
            .shuffle(buffer_size)
            .batch(batch_size, drop_remainder=True)
            .cache()
            .prefetch(tf.data.experimental.AUTOTUNE))

### Build the model

If the model can be restored from the saved file, use that.  Otherwise, build and train the model

In [ ]:
%%time

try:
  model = tf.keras.models.load_model(model_save_file)
except:
  # definition for the inputs.  Note how num of inputs is realted to size of key_order global
  input_shape = (seq_length, len(key_order))
  learning_rate = 0.010

  #input layer
  inputs = tf.keras.Input(input_shape)
  #hidden layers
  x = tf.keras.layers.LSTM(128, return_sequences=True)(inputs)
  x = tf.keras.layers.Dropout(0.20, seed=seed)(x) #dropout layer
  x = tf.keras.layers.LSTM(16, return_sequences=True)(x)
  x = tf.keras.layers.Dropout(0.20, seed=seed)(x) #dropout layer
  x = tf.keras.layers.LSTM(16, return_sequences=False)(x) #last layer to outputs




  outputs = {
    'pitch': tf.keras.layers.Dense(128, name='pitch')(x),
    'step': tf.keras.layers.Dense(1, name='step')(x),
    'duration': tf.keras.layers.Dense(1, name='duration')(x),
  }

  model = tf.keras.Model(inputs, outputs)

  loss = {
        'pitch':tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        'step': "mean_squared_error",
        'duration': "mean_squared_error",
  }

  # loss = {
  #       'pitch':tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  #       'step': mse_with_positive_pressure,
  #       'duration': mse_with_positive_pressure,
  # }

  optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

  model.compile(loss=loss, optimizer=optimizer)

  model.summary()
  # these are the three outputs of the model

  model.compile(
      loss=loss,
      loss_weights={
          'pitch': 1.0,
          'step': 1.0,
          'duration':1.0,
      },
      optimizer=optimizer,
  )

  model.evaluate(train_ds, return_dict=True)
  callbacks = [
      tf.keras.callbacks.ModelCheckpoint(
          filepath='./training_checkpoints/ckpt_{epoch}.weights.h5',
          save_weights_only=True),
      tf.keras.callbacks.EarlyStopping(
          monitor='loss',
          min_delta=0.001,
          patience=5,
          verbose=1,
          start_from_epoch=2,
          restore_best_weights=True),
  ]
  
  epochs = 30

  history = model.fit(
      train_ds,
      epochs=epochs,
      callbacks=callbacks,
      validation_data=val_ds,
  )
  model.save(model_save_file)
  np.save('history_purepitch.npy', history.history, allow_pickle=True) # save training data for later

In [ ]:
loaded_history = np.load('history_purepitch.npy', allow_pickle=True) # save training data for later
loaded_history.item(0)['loss']

In [ ]:
def predict_next_note(
    notes: np.ndarray,
    model: tf.keras.Model,
    temperature: float = 1.0) -> tuple[int, float, float]:
  """Generates a note as a tuple of (pitch, step, duration), using a trained sequence model."""

  assert temperature > 0

  # Add batch dimension
  inputs = tf.expand_dims(notes, 0)

  predictions = model.predict(inputs)
  pitch_logits = predictions['pitch']
  step = predictions['step']
  duration = predictions['duration']

  pitch_logits /= temperature 
  pitch = tf.random.categorical(pitch_logits, num_samples=1) 
  pitch = tf.squeeze(pitch, axis=-1) 
  duration = tf.squeeze(duration, axis=-1)
  step = tf.squeeze(step, axis=-1)

  # `step` and `duration` values should be non-negative

  step = tf.maximum(0, step)
  duration = tf.maximum(0, duration)
  

  return int(pitch), float(step), float(duration), pitch_logits

In [ ]:
def calc_unit_perplexity(logits, target): 
  metric=Perplexity()
  input = torch.tensor(logits)
  target_tens = torch.tensor([target])
  metric.update(input, target_tens)
  return metric.compute()

#calc_unit_perplexity(agg_logits[0],)

In [ ]:
temperature = 1.0
num_predictions = seq_length

def note_loop(song):
  test_notes = pitchInst[pitchInst['melid'] == song].reset_index()
  sample_notes = np.stack([test_notes[key] for key in key_order], axis=1)

  input_notes = sample_notes[:seq_length]

  # These are common fields to the entire test set
  tempo = test_notes['tempo'].iloc[seq_length]
  inst = test_notes['instrument_num'].iloc[seq_length]
  key = test_notes['key_num'].iloc[seq_length]
  title = test_notes['title'].iloc[seq_length]
  performer = test_notes['performer'].iloc[seq_length]
  instrument_name = test_notes['instrument'].iloc[seq_length]

  generated_notes = []
  # this is the last start time of the prompt set
  prev_start = test_notes['start'][seq_length]
  agg_logits = []
  pplex = []
  #contour = 0  #initial contour should be last contour of input sequence
  for i in range(num_predictions): #THERE ARE PROBLEMS HERE
    pitch, step, duration, pitch_logits = predict_next_note(input_notes, model, temperature)
    start = prev_start
    end = start + duration
    interval = input_notes[-1][0] - pitch
    contour = my_functions.contour(interval)
    agg_logits.append(pitch_logits)
    target_note = test_notes['pitch'].iloc[seq_length + i].astype(int)
    pplex.append(calc_unit_perplexity([pitch_logits], [target_note]))
    # TODO:
    # This line has to change when you change the inputs to the model.  The input_note
    # that is getting appended to input notes needs to have the correct number and order of 
    # fields cause it is gonna get fed back into the model.predict function
    # input_note = (pitch, step, duration)
    input_note = (pitch, step, duration, tempo, inst, key)
    generated_notes.append((*input_note, start, end))
    input_notes = np.delete(input_notes, 0, axis=0)
    input_notes = np.append(input_notes , np.expand_dims(input_note, 0), axis=0) 
    prev_start = start + step
    
  generated_notes = pd.DataFrame(
      generated_notes, columns=(*key_order, 'start', 'end'))

  start_df = test_notes[:seq_length].drop(['interval'], axis=1)
  

  # string together the first training data and the generated notes
  full_sequence = pd.concat([start_df, generated_notes], ignore_index=True)

  example_file = f"MixedModel-Song-{song}_seq-{seq_length}-{performer}-{title}.midi"  # adds a prefix to the sample filename
  my_functions.notes_to_midi(full_sequence[['pitch', 'step', 'duration', 'tempo']], out_file=example_file, instrument_name='Acoustic Grand Piano')

  return test_notes[:len(full_sequence)], full_sequence, generated_notes, title, performer, agg_logits, pplex


In [ ]:
run_list = test_list
#run_list = [41, 42, 74, 75, 89]
targets, full_sequence, generated_notes, title, performer, agg_logits, pplex = note_loop(run_list[1])


In [ ]:
import matplotlib.pyplot as plt

# Plot pplex as a line graph
plt.figure(figsize=(10, 6))
plt.plot(pplex, marker='o', linestyle='-', color='b')
plt.title('Perplexity Over Time')
plt.xlabel('Prediction Step')
plt.ylabel('Perplexity')
plt.grid(True)
plt.show()

In [ ]:
def calc_perplexity(agg_logits, target, num_predictions):
  x = target['pitch'].tail(num_predictions).astype(int).to_numpy()
  np.array_split(x, num_predictions)
  metric=Perplexity()
  input = torch.tensor(agg_logits)
  target_tens = torch.tensor(np.array_split(x, num_predictions))
  metric.update(input, target_tens)
  return metric.compute()

In [ ]:
calc_perplexity(agg_logits, targets, num_predictions)

In [ ]:
my_functions.plot_notes(full_sequence, seq_length ,heading=f'Predictions: {title}-{performer}')
my_functions.plot_notes(targets, seq_length, heading=f'Actual: {title}-{performer}')

In [ ]:
my_functions.plot_distributions(generated_notes, title="Generated Notes Distribution")
my_functions.plot_distributions(targets, title="Target Data Notes")